<a href="https://colab.research.google.com/github/Skromos/Steganografia-F-bio/blob/main/ProjetodaDisciplina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.backends import default_backend
from base64 import b64encode, b64decode
import hashlib
import os

# Definição de estilos de cores para tornar a interface mais amigável
def print_divider():
    print("\033[94m" + "=" * 50 + "\033[0m")  # Linha azul

def print_title(title):
    print("\033[96m" + f"\n{title}\n" + "\033[0m")

def print_info(message):
    print("\033[93m" + message + "\033[0m")  # Amarelo para instruções

def print_success(message):
    print("\033[92m" + message + "\033[0m")  # Verde para sucesso

def print_error(message):
    print("\033[91m" + message + "\033[0m")  # Vermelho para erro

def print_credits():
    print("\033[94m\033[3mDesenvolvido por Dev Baraus e Luiz Felipe\033[0m")  # Créditos em azul e itálico

# Funções principais de criptografia e manipulação de imagem
def generate_rsa_key_pair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    public_key = private_key.public_key()
    return private_key, public_key

def save_keys_to_files(private_key, public_key):
    with open("private_key.pem", "wb") as private_file:
        private_file.write(
            private_key.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.TraditionalOpenSSL,
                encryption_algorithm=serialization.NoEncryption()
            )
        )

    with open("public_key.pem", "wb") as public_file:
        public_file.write(
            public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
        )

def encrypt_with_public_key(public_key, message):
    encrypted = public_key.encrypt(
        message.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return b64encode(encrypted).decode()

def decrypt_with_private_key(private_key, encrypted_message):
    decrypted = private_key.decrypt(
        b64decode(encrypted_message),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return decrypted.decode()

def load_private_key(file_path):
    with open(file_path, "rb") as private_file:
        private_key = serialization.load_pem_private_key(
            private_file.read(),
            password=None,
            backend=default_backend()
        )
    return private_key

def load_public_key(file_path):
    with open(file_path, "rb") as public_file:
        public_key = serialization.load_pem_public_key(
            public_file.read(),
            backend=default_backend()
        )
    return public_key

def text_to_bin(text):
    return ''.join([format(ord(char), '08b') for char in text])

def hide_text_in_image(image_path, text, output_image_path):
    try:
        image = Image.open(image_path)
        if image.format not in ["JPEG", "PNG"]:
            print_error("Erro ao inserir imagem, formato inválido.")
            return False

        if image.mode != 'RGB':
            image = image.convert('RGB')

        encoded_image = image.copy()
        pixels = list(encoded_image.getdata())

        binary_text = text_to_bin(text) + text_to_bin('###')
        binary_index = 0

        for i in range(len(pixels)):
            if binary_index < len(binary_text):
                pixel = list(pixels[i])
                for j in range(3):
                    if binary_index < len(binary_text):
                        pixel[j] = pixel[j] & ~1 | int(binary_text[binary_index])
                        binary_index += 1
                pixels[i] = tuple(pixel)
            else:
                break

        encoded_image.putdata(pixels)
        encoded_image.save(output_image_path)
        print_success("Texto escondido na imagem com sucesso!")
        return True
    except Exception:
        print_error("Erro ao inserir imagem, caminho ou formato inválidos.")
        return False

def bin_to_text(binary_data):
    text = ''
    for i in range(0, len(binary_data), 8):
        byte = binary_data[i:i+8]
        char = chr(int(byte, 2))
        if char == '#':
            break
        text += char
    return text

def extract_text_from_image(image_path):
    image = Image.open(image_path)
    pixels = list(image.getdata())
    binary_data = ''

    for pixel in pixels:
        for channel in pixel[:3]:
            binary_data += str(channel & 1)

    extracted_text = bin_to_text(binary_data)
    return extracted_text

def sha1_hash_file(file_path):
    sha1 = hashlib.sha1()
    with open(file_path, 'rb') as file:
        while True:
            data = file.read(4096)
            if not data:
                break
            sha1.update(data)
    return sha1.hexdigest()

# Função para continuar ou encerrar o programa
def continue_or_exit():
    while True:
        print("\033[93mDeseja continuar? (S para continuar / N para encerrar)\033[0m")
        choice = input().strip().lower()
        if choice == 's':
            return True
        elif choice == 'n':
            print_success("\nPrograma encerrado. Até logo!")
            return False
        else:
            print_error("Opção inválida. Por favor, digite 'S' ou 'N'.")

# Função para mostrar o tutorial de como inserir imagens no Google Colab
def tutorial_inserir_imagens():
    print("\n\033[93mTutorial: Como inserir imagens no Google Colab\033[0m")
    print("1. Primeiro, faça o upload da sua imagem no Google Colab:")
    print("   - Vá até a barra lateral à esquerda e clique no icone de uma pasta chamada 'Arquivos'.")
    print("   - Clique em 'Upload' para selecionar a imagem no seu computador.")
    print("2. Após o upload, clique com o botão direito no arquivo e copie o diretório .")
    print("   - O caminho geralmente será algo como '/content/nome_da_imagem.png'.")
    print("3. Use esse caminho para inserir a imagem em nosso programa.\n")
    print("\033[92mAgora você pode continuar com o programa!\033[0m")

# Função principal
def main():
    # Pergunta inicial para o tutorial
    print_title("🔐 Bem-vindo ao StegoCrypt nosso programa de Criptografia e Esteganografia 🔐")
    print("\033[93mGostaria de saber como inserir imagens no Google Colab?\033[0m")
    print("Digite 'S' para ver o tutorial ou 'N' para pular e continuar.")

    while True:
        tutorial_choice = input().strip().lower()
        if tutorial_choice == 's':
            tutorial_inserir_imagens()
            break
        elif tutorial_choice == 'n':
            break
        else:
            print_error("Opção inválida. Por favor, digite 'S' ou 'N'.")

    # Carrega ou gera chaves
    try:
        private_key = load_private_key('private_key.pem')
        public_key = load_public_key('public_key.pem')
        print_success("🔑 Chaves RSA carregadas com sucesso!")
    except:
        private_key, public_key = generate_rsa_key_pair()
        save_keys_to_files(private_key, public_key)
        print_success("🔑 Chaves RSA geradas e salvas com sucesso!")

    print_divider()

    # Recebe o texto e caminho da imagem do usuário
    print_info("Digite o texto que você deseja criptografar e esconder na imagem.")
    message = input("📝 Texto: ")

    print_divider()

    # Loop para garantir um caminho de imagem válido
    while True:
        print_info("Digite o caminho da imagem (JPEG ou PNG) onde o texto será embutido.")
        image_path = input("📁 Caminho da imagem: ")
        output_image_path = 'output.png'

        encrypted_message = encrypt_with_public_key(public_key, message)
        if hide_text_in_image(image_path, encrypted_message, output_image_path):
            print_success("✅ Imagem inserida com sucesso!")
            break

    print_divider()

    # Mostra o texto inserido pelo usuário e o texto criptografado
    print(f"📝 Texto inserido pelo usuário: {message}")
    print(f"🔒 Texto criptografado: {encrypted_message}")

    # Gera hashes das imagens de entrada e saída
    hash_original = sha1_hash_file(image_path)
    hash_modificada = sha1_hash_file(output_image_path)

    print("\033[94m\n📊 Hashes das imagens 📊\033[0m")
    print(f"🔹 Hash da imagem original: {hash_original}")
    print(f"🔹 Hash da imagem modificada: {hash_modificada}")

    print_divider()

    # Extrai e decriptografa o texto
    extracted_text = extract_text_from_image(output_image_path)
    decrypted_message = decrypt_with_private_key(private_key, extracted_text)

    print("\033[94m\n📜 Resultados da Extração e Decriptação 📜\033[0m")
    print(f"📥 Texto extraído da imagem: {extracted_text}")
    print(f"🔓 Texto descriptografado: {decrypted_message}")

    print_divider()
    print_success("🎉 Processo concluído com sucesso! Obrigado por usar o programa. 🎉")

    # Exibe os créditos
    print_credits()

    # Pergunta se o usuário deseja continuar
    if continue_or_exit():
        main()

# Inicia o programa
if __name__ == "__main__":
    main()



🔐 Bem-vindo ao StegoCrypt nosso programa de Criptografia e Esteganografia 🔐

Gostaria de saber como inserir imagens no Google Colab?
Digite 'S' para ver o tutorial ou 'N' para pular e continuar.
🔑 Chaves RSA carregadas com sucesso!
Digite o texto que você deseja criptografar e esconder na imagem.
Digite o caminho da imagem (JPEG ou PNG) onde o texto será embutido.
Erro ao inserir imagem, caminho ou formato inválidos.
Digite o caminho da imagem (JPEG ou PNG) onde o texto será embutido.
Erro ao inserir imagem, caminho ou formato inválidos.
Digite o caminho da imagem (JPEG ou PNG) onde o texto será embutido.
Texto escondido na imagem com sucesso!
✅ Imagem inserida com sucesso!
📝 Texto inserido pelo usuário: teste teste
🔒 Texto criptografado: mpK++ZtwqwWt99zC+A8yinfMBZowjgCUQUhl8ko8sPKs7bXdDt4NFkQhutg4R+172ys06B5oPpbfBXMu1W/xhE8hUg2EwMPoo2vyFdbJY0aipfdQdAbNNa6JW+gX5PNLA6PTNSBQpzmLT4mvJPuSoSnOLkQ5OQJwkVqlmStA+JYHf7DflvG9dJp7ujanekkS6XH030GFEpNKPpM4PzgMcJq9ZQudTFUs0j9nY3lFRHw0owBbuWEmOTyEA5